In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from deduping_module import deduping_class

In [2]:
link = r'C:\Users\USER\Documents\LM_project\SAL-230_Revisit validity'
gt = pd.read_csv(link + '\\raw\sf_export.csv', encoding='latin-1')
nm = pd.read_csv(link + '\\raw\ohio-00001 - step 1.csv', encoding='latin-1')

In [3]:
gt

,Salesforce Account Id,Salesforce Contact Id,Account Name,First Name,Last Name,Email,Phone,Street Address 1,Zip Code 1,County 1,State 1,Quarantine
0,0014y00002LvzGCAAZ,0034y00002QqcZtAAJ,Melcher Law,Doug,Melcher,NaN,(617) 485-0859,NaN,NaN,NaN,NaN,0.0
1,0014y00002aVeATAA0,0034y00002ZjgTZAAZ,Barbara Wolf,Barbara,Wolf,NaN,(513) 772-6887,102 Hetherington Lane,NaN,NaN,OH,0.0
2,0014y00002ckHrlAAE,0034y00002ZjrLLAAZ,Gerard Haag,Gerard,Haag,NaN,(703) 692-9265,7108 Vermillion Pl,NaN,NaN,VA,0.0
3,0014y00002aVbH8AAK,0034y00002ZjdWdAAJ,Jeremiah Webb,Jeremiah,Webb,NaN,(865) 288-7866,6812 Sherwood Drive,NaN,NaN,TN,0.0
4,0014y00002aVSjaAAG,0034y00002ZjUyfAAF,John Hodge,John,Hodge,jehodge@jehodgelaw.com,NaN,2950 Sharon View Road,NaN,NaN,NC,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1606613,All contacts and account,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606614,"Copyright (c) 2000-2022 salesforce.com, inc. A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606615,Confidential Information - Do Not Distribute,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606616,Generated By: Raffie Navaluna 10/4/2022 4:17 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
gt.dropna(subset= ['Salesforce Contact Id'], inplace= True)
gt.drop_duplicates(subset=['Salesforce Account Id'], inplace= True) # need to remove the duplicates in the salesforce account id if we dedupe on the account object
gt.reset_index(drop=True, inplace= True) #always make sure that the index of the inputs are in numerical order or this will cause errors in getting the matches
gt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095681 entries, 0 to 1095680
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Salesforce Account Id  1095681 non-null  object 
 1   Salesforce Contact Id  1095681 non-null  object 
 2   Account Name           1095680 non-null  object 
 3   First Name             1094433 non-null  object 
 4   Last Name              1095656 non-null  object 
 5   Email                  622569 non-null   object 
 6   Phone                  1061305 non-null  object 
 7   Street Address 1       1023387 non-null  object 
 8   Zip Code 1             870768 non-null   object 
 9   County 1               820987 non-null   object 
 10  State 1                1069961 non-null  object 
 11  Quarantine             1095681 non-null  float64
dtypes: float64(1), object(11)
memory usage: 100.3+ MB


Note:
1. the footer on the ground truth file should always be dropped

In [5]:
# initializing the module
deduping = deduping_class(gt, 'account') 

In [6]:
deduping.ground_truth.head()

,Salesforce Account Id,Salesforce Contact Id,Account Name,First Name,Last Name,Email,Phone,Street Address 1,Zip Code 1,County 1,State 1,Quarantine
0,0014y00002LvzGCAAZ,0034y00002QqcZtAAJ,Melcher Law,Doug,Melcher,NaN,(617) 485-0859,NaN,NaN,NaN,NaN,0.0
1,0014y00002aVeATAA0,0034y00002ZjgTZAAZ,Barbara Wolf,Barbara,Wolf,NaN,(513) 772-6887,102 Hetherington Lane,NaN,NaN,OH,0.0
2,0014y00002ckHrlAAE,0034y00002ZjrLLAAZ,Gerard Haag,Gerard,Haag,NaN,(703) 692-9265,7108 Vermillion Pl,NaN,NaN,VA,0.0
3,0014y00002aVbH8AAK,0034y00002ZjdWdAAJ,Jeremiah Webb,Jeremiah,Webb,NaN,(865) 288-7866,6812 Sherwood Drive,NaN,NaN,TN,0.0
4,0014y00002aVSjaAAG,0034y00002ZjUyfAAF,John Hodge,John,Hodge,jehodge@jehodgelaw.com,NaN,2950 Sharon View Road,NaN,NaN,NC,0.0


#### Step 1:
1. Make sure that the ground truth columns are similar with the dataframe to be matched

In [7]:
# matching the ground truth columns with the to match dataframe
nm.rename(columns={'firstName':'First Name', 'lastName':'Last Name', 'email':'Email', 'phone':'Phone', 'zip':'Zip Code 1', 'First and Last':'Account Name', 'street':'Street Address 1', 'state':'State 1'}, inplace= True)

In [8]:
nm.head(3)

,Account Name,First Name,Last Name,suffix,status,Phone,phoneExt,fax,Web,Street Address 1,city,State 1,Zip Code 1,Email,Bar Admission Date,Bar State
0,Judith Gliniecki,Judith,Gliniecki,NaN,Active,482-256-4551,NaN,NaN,NaN,ul. Grzybowska 5A,Warsaw,,NaN,NaN,11/18/1991,OH
1,Maria Abe,Maria,Abe,NaN,Active,813-686-8840,NaN,NaN,NaN,Shin-Tokyo Building 2F-246,Chiyoda-ku,,100,NaN,5/9/2011,OH
2,Tsuyoshi Ishida,Tsuyoshi,Ishida,NaN,Active,033-221-2455,NaN,NaN,NaN,"1-1-1 Hirakawacho, Chiyoda-ku",Tokyo,,102,NaN,11/8/1993,OH


In [9]:
nm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32490 entries, 0 to 32489
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Account Name        32490 non-null  object 
 1   First Name          32490 non-null  object 
 2   Last Name           32490 non-null  object 
 3   suffix              1530 non-null   object 
 4   status              32490 non-null  object 
 5   Phone               32490 non-null  object 
 6   phoneExt            158 non-null    float64
 7   fax                 0 non-null      float64
 8   Web                 0 non-null      float64
 9   Street Address 1    32461 non-null  object 
 10  city                32476 non-null  object 
 11  State 1             32490 non-null  object 
 12  Zip Code 1          32477 non-null  object 
 13  Email               0 non-null      float64
 14  Bar Admission Date  32127 non-null  object 
 15  Bar State           32490 non-null  object 
dtypes: f

#### Step 2
Now that the columns are matched, we can use the key_selector function. \
This takes 2 inputs, *args and "data=dataframe to be matched"\
This function will return the to be matched dataframe with primary_key value while also adding it in the ground truth but only in the backend

In [10]:
deduping.key_selector('Account Name', 'Street Address 1', 'State 1', 'Phone', data= nm)

In [11]:
# deduping.nm['State 1'] = deduping.nm.apply(lambda x: deduping.state_abbrev(x['State 1']), axis= 1)

#### Optional Step
We can set an optional paramater called ngrams, this means the number of combination the txt will be divided. \
if this is not set, it will automatically equal to 3

In [12]:
deduping.set_ngrams(4)

#### Step 3
This function will convert the ground truth and dataframe to be matched in to tfidf sparse matrix \
the input to this function should be the dataframe with the primarykey \
the results will now be included in the class module

self.nm_tfidf\
self.gt_tfidf

In [13]:
deduping.vectorizer()

In [14]:
deduping.nm_tfidf_df

,\t\n34,\t\nSu,\t#20,\t107,\t120,\t134,\t212,\t21M,\t22n,\t235,...,öm15,ömTr,øren,úaJB,únio,úsSa,üell,üero,ühr),üst1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
deduping.gt_tfidf_df

,\t\n34,\t\nSu,\t#20,\t107,\t120,\t134,\t212,\t21M,\t22n,\t235,...,öm15,ömTr,øren,úaJB,únio,úsSa,üell,üero,ühr),üst1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1095677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1095678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1095679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Step 4
This function will get compare the similarities in the 2 dataframe and save the output in self.match

In [16]:
deduping.get_match(1000000)

The top value is not set or the value exceeds the nonzero size


In [17]:
deduping.matched

,index,Matched DataFrame Key,Ground Truth Key,Ground Truth ID,similarity
0,0,JudithGlinieckiul.Grzybowska5A482-256-4551,JudithGlinieckiul.Grzybowska5Anan482-256-4551,0034y00002hZlDpAAK,0.850367
1,1,MariaAbeShin-TokyoBuilding2F-246813-686-8840,MariaAbeShin-TokyoBuilding2F-246nan813-686-8840,0034y00002hZlDqAAK,0.879594
2,2,"TsuyoshiIshida1-1-1Hirakawacho,Chiyoda-ku033-2...","TsuyoshiIshida1-1-1Hirakawacho,Chiyoda-kuOH033...",0034y00002ZjgHiAAJ,0.860880
3,3,"DanielBurkhartArkHillSengokuyamaMoriTower,28th...","DanielBurkhartArkHillSengokuyamaMoriTower,28th...",0034y00002hZlDrAAK,0.901561
4,4,MasashiOkuyamaIzumiGardenTower19F813-622-9012,MasashiOkuyamaIzumiGardenTower19Fnan813-622-9012,0034y00002hZlDsAAK,0.847645
...,...,...,...,...,...
16692,32483,MarthaHudak3300OliveAvenueCA818-977-2500,MarthaHudak3300OliveAvenueCA818-977-2500,0034y00002hZnM3AAK,1.000000
16693,32484,TanyaHanna1HackerWayCA650-853-1300,TanyaHanna1HackerWayCA650-853-1300,0034y00002hZnM4AAK,1.000000
16694,32485,BenjaminDickens595MarketStreetCA919-260-5151,BenjaminDickens595MarketStreetCA919-260-5151,0034y00002hZnM5AAK,1.000000
16695,32486,"SafiyahSpann525MarketStreet,15thFloorCA415-601...","SafiyahSpann525MarketStreet,15thFloorCA(415)60...",0034y00002ZjcLXAAZ,0.860641


#### Notes:
1. If deduping at account object, always remember to remove the duplicates in the Salesforce Account Id
1. always make sure that the index of the inputs are in numerical order or this will cause errors in getting the matches
1. if a selected key value is missing, it has a significant impact on the performance

In [18]:
stopper

NameError: name 'stopper' is not defined

#### Performance Evaluation

In [19]:
detected_duplicates = nm.merge(deduping.matched[['index','Ground Truth ID','similarity']].set_index('index'), left_index= True, right_index=True)

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [20]:
len(detected_duplicates) - len(detected_duplicates[detected_duplicates['similarity'] == 1])

15062

In [21]:
len(detected_duplicates.drop_duplicates(subset=['Account Name'])) - len(detected_duplicates[detected_duplicates['similarity'] == 1].drop_duplicates(subset=['Account Name']))

14788

In [22]:
detected_duplicates.loc[5, 'primary_key']

"TannerLawrence1000Blackbeard'sHillVI614-939-9955"

In [23]:
detected_duplicates

,Account Name,First Name,Last Name,suffix,status,Phone,phoneExt,fax,Web,Street Address 1,city,State 1,Zip Code 1,Email,Bar Admission Date,Bar State,primary_key,Ground Truth ID,similarity
0,JudithGliniecki,Judith,Gliniecki,NaN,Active,482-256-4551,NaN,NaN,NaN,ul.Grzybowska5A,Warsaw,,NaN,NaN,11/18/1991,OH,JudithGlinieckiul.Grzybowska5A482-256-4551,0034y00002hZlDpAAK,0.850367
1,MariaAbe,Maria,Abe,NaN,Active,813-686-8840,NaN,NaN,NaN,Shin-TokyoBuilding2F-246,Chiyoda-ku,,100,NaN,5/9/2011,OH,MariaAbeShin-TokyoBuilding2F-246813-686-8840,0034y00002hZlDqAAK,0.879594
2,TsuyoshiIshida,Tsuyoshi,Ishida,NaN,Active,033-221-2455,NaN,NaN,NaN,"1-1-1Hirakawacho,Chiyoda-ku",Tokyo,,102,NaN,11/8/1993,OH,"TsuyoshiIshida1-1-1Hirakawacho,Chiyoda-ku033-2...",0034y00002ZjgHiAAJ,0.860880
3,DanielBurkhart,Daniel,Burkhart,NaN,Active,813-627-1975,NaN,NaN,NaN,"ArkHillSengokuyamaMoriTower,28thFloor",Tokyo,OH,106,NaN,7/9/2008,OH,"DanielBurkhartArkHillSengokuyamaMoriTower,28th...",0034y00002hZlDrAAK,0.901561
4,MasashiOkuyama,Masashi,Okuyama,NaN,Active,813-622-9012,NaN,NaN,NaN,IzumiGardenTower19F,Minato ku,,106,NaN,11/5/2012,OH,MasashiOkuyamaIzumiGardenTower19F813-622-9012,0034y00002hZlDsAAK,0.847645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32483,MarthaHudak,Martha,Hudak,NaN,Corporate,818-977-2500,NaN,NaN,NaN,3300OliveAvenue,Burbank,CA,91505,NaN,NaN,OH,MarthaHudak3300OliveAvenueCA818-977-2500,0034y00002hZnM3AAK,1.000000
32484,TanyaHanna,Tanya,Hanna,NaN,Corporate,650-853-1300,NaN,NaN,NaN,1HackerWay,Menlo Park,CA,94025,NaN,NaN,OH,TanyaHanna1HackerWayCA650-853-1300,0034y00002hZnM4AAK,1.000000
32485,BenjaminDickens,Benjamin,Dickens,NaN,Corporate,919-260-5151,NaN,NaN,NaN,595MarketStreet,San Francisco,CA,94105,NaN,NaN,OH,BenjaminDickens595MarketStreetCA919-260-5151,0034y00002hZnM5AAK,1.000000
32486,SafiyahSpann,Safiyah,Spann,NaN,Corporate,415-601-3945,NaN,NaN,NaN,"525MarketStreet,15thFloor",San Francisco,CA,94105,NaN,NaN,OH,"SafiyahSpann525MarketStreet,15thFloorCA415-601...",0034y00002ZjcLXAAZ,0.860641


In [24]:
Stopper

NameError: name 'Stopper' is not defined

In [25]:
import glob

demandtools_output = glob.glob(link +'\\raw\\*step 3*')
demandtools_output = [value for value in demandtools_output if 'Non_Match' not in value]

In [26]:
temp_list = []
for file in demandtools_output:
    temp_df = pd.read_csv(file, low_memory=False)
    temp_list.append(temp_df)
demandtools_output = pd.concat(temp_list)

In [27]:
demandtools_output.head(2)

,First and Last,First Name,Last Name,suffix,status,Phone,phoneExt,fax,Web,street,...,Match_ShippingState,Match_ShippingStreet,Match_ShippingPostalCode,Match_Status__c,Match_SystemModstamp,Match_TickerSymbol,Match_Waitlist__c,Match_Website,Match_Zip_Code_2__c,Match_Zip_Code_3__c
0,Michael Magee,Michael,Magee,NaN,Active,412-288-5870,NaN,NaN,NaN,1300 Six PPG Place,...,NaN,NaN,NaN,Prospect,2022-07-01T17:06:04.000+0000,NaN,NaN,NaN,NaN,NaN
1,Michael Magee,Michael,Magee,NaN,Active,412-288-5870,NaN,NaN,NaN,1300 Six PPG Place,...,NaN,NaN,NaN,Prospect,2022-04-03T08:02:09.000+0000,NaN,NaN,NaN,NaN,NaN


In [28]:
len(demandtools_output['First and Last'].unique())

12504

In [29]:
detected_duplicates[~detected_duplicates['Account Name'].isin(demandtools_output['First and Last'].unique().tolist())].drop_duplicates(subset=['Account Name'])

,Account Name,First Name,Last Name,suffix,status,Phone,phoneExt,fax,Web,Street Address 1,city,State 1,Zip Code 1,Email,Bar Admission Date,Bar State,primary_key,Ground Truth ID,similarity
0,JudithGliniecki,Judith,Gliniecki,NaN,Active,482-256-4551,NaN,NaN,NaN,ul.Grzybowska5A,Warsaw,,NaN,NaN,11/18/1991,OH,JudithGlinieckiul.Grzybowska5A482-256-4551,0034y00002hZlDpAAK,0.850367
1,MariaAbe,Maria,Abe,NaN,Active,813-686-8840,NaN,NaN,NaN,Shin-TokyoBuilding2F-246,Chiyoda-ku,,100,NaN,5/9/2011,OH,MariaAbeShin-TokyoBuilding2F-246813-686-8840,0034y00002hZlDqAAK,0.879594
2,TsuyoshiIshida,Tsuyoshi,Ishida,NaN,Active,033-221-2455,NaN,NaN,NaN,"1-1-1Hirakawacho,Chiyoda-ku",Tokyo,,102,NaN,11/8/1993,OH,"TsuyoshiIshida1-1-1Hirakawacho,Chiyoda-ku033-2...",0034y00002ZjgHiAAJ,0.860880
3,DanielBurkhart,Daniel,Burkhart,NaN,Active,813-627-1975,NaN,NaN,NaN,"ArkHillSengokuyamaMoriTower,28thFloor",Tokyo,OH,106,NaN,7/9/2008,OH,"DanielBurkhartArkHillSengokuyamaMoriTower,28th...",0034y00002hZlDrAAK,0.901561
4,MasashiOkuyama,Masashi,Okuyama,NaN,Active,813-622-9012,NaN,NaN,NaN,IzumiGardenTower19F,Minato ku,,106,NaN,11/5/2012,OH,MasashiOkuyamaIzumiGardenTower19F813-622-9012,0034y00002hZlDsAAK,0.847645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32483,MarthaHudak,Martha,Hudak,NaN,Corporate,818-977-2500,NaN,NaN,NaN,3300OliveAvenue,Burbank,CA,91505,NaN,NaN,OH,MarthaHudak3300OliveAvenueCA818-977-2500,0034y00002hZnM3AAK,1.000000
32484,TanyaHanna,Tanya,Hanna,NaN,Corporate,650-853-1300,NaN,NaN,NaN,1HackerWay,Menlo Park,CA,94025,NaN,NaN,OH,TanyaHanna1HackerWayCA650-853-1300,0034y00002hZnM4AAK,1.000000
32485,BenjaminDickens,Benjamin,Dickens,NaN,Corporate,919-260-5151,NaN,NaN,NaN,595MarketStreet,San Francisco,CA,94105,NaN,NaN,OH,BenjaminDickens595MarketStreetCA919-260-5151,0034y00002hZnM5AAK,1.000000
32486,SafiyahSpann,Safiyah,Spann,NaN,Corporate,415-601-3945,NaN,NaN,NaN,"525MarketStreet,15thFloor",San Francisco,CA,94105,NaN,NaN,OH,"SafiyahSpann525MarketStreet,15thFloorCA415-601...",0034y00002ZjcLXAAZ,0.860641


In [30]:
demandtools_output[~demandtools_output['First and Last'].isin(detected_duplicates['Account Name'].unique().tolist())].drop_duplicates(subset=['First and Last'])

,First and Last,First Name,Last Name,suffix,status,Phone,phoneExt,fax,Web,street,...,Match_ShippingState,Match_ShippingStreet,Match_ShippingPostalCode,Match_Status__c,Match_SystemModstamp,Match_TickerSymbol,Match_Waitlist__c,Match_Website,Match_Zip_Code_2__c,Match_Zip_Code_3__c
0,Michael Magee,Michael,Magee,NaN,Active,412-288-5870,NaN,NaN,NaN,1300 Six PPG Place,...,NaN,NaN,NaN,Prospect,2022-07-01T17:06:04.000+0000,NaN,NaN,NaN,NaN,NaN
2,Jeffrey Paravano,Jeffrey,Paravano,NaN,Active,202-861-1770,NaN,NaN,NaN,1050 Connecticut Ave NW,...,NaN,NaN,NaN,Prospect,2022-04-03T08:03:44.000+0000,NaN,NaN,NaN,NaN,NaN
4,Glen Garrett,Glen,Garrett,NaN,Active,216-310-0838,NaN,NaN,NaN,1357 Sassafrass Circle,...,NaN,NaN,NaN,Prospect,2022-08-19T13:51:31.000+0000,NaN,NaN,NaN,NaN,NaN
6,Michael Mutek,Michael,Mutek,NaN,Active,256-971-3562,NaN,NaN,NaN,950 Explorer Blvd,...,NaN,NaN,NaN,Prospect,2022-04-03T08:01:54.000+0000,NaN,NaN,NaN,NaN,NaN
8,Erica Olivares-Vincent,Erica,Olivares-Vincent,NaN,Active,303-727-0178,NaN,NaN,NaN,895 N. 6th St.,...,NaN,NaN,NaN,Prospect,2022-03-28T08:13:52.000+0000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12605,Justin Relihan,Justin,Relihan,NaN,Corporate,314-595-8060,NaN,NaN,NaN,8050 W. Florissant Ave,...,NaN,NaN,NaN,Prospect,2022-03-28T09:28:30.000+0000,NaN,NaN,NaN,NaN,NaN
12606,Erin Ryan,Erin,Ryan,NaN,Corporate,877-743-2269,NaN,NaN,NaN,8151 Airline Officer,...,NaN,NaN,NaN,Prospect,2022-03-28T09:55:29.000+0000,NaN,NaN,NaN,NaN,NaN
12607,Michael DiSanto,Michael,DiSanto,NaN,Corporate,602-850-4821,NaN,NaN,NaN,1455 N. Scottsdale Rd,...,NaN,NaN,NaN,Prospect,2022-03-28T08:31:42.000+0000,NaN,NaN,NaN,NaN,NaN
12608,Safiyah Spann,Safiyah,Spann,NaN,Corporate,415-601-3945,NaN,NaN,NaN,"525 Market Street, 15th Floor",...,NaN,NaN,NaN,Prospect,2022-03-28T09:14:41.000+0000,NaN,NaN,NaN,NaN,NaN


In [31]:
test = detected_duplicates.merge(demandtools_output[['First and Last']].drop_duplicates(subset=['First and Last']), how='inner', left_on='Account Name', right_on='First and Last')